Vamos a desglosar un poco del codigo y vamos a ir pasando lo a pytorch para trabajar mas a detalle:
```python

qkm_model = QKMClassModel(encoded_size=encoded_size,
                            dim_y=dim_y,
                            encoder=encoder,
                            n_comp=n_comp,
                            sigma=0.1)
```
Función de llamada del QKMclasModel

```python
def call(self, input):
    encoded = self.encoder(input)
    rho_x = pure2dm(encoded)
    rho_y =self.qkm(rho_x)
    probs = dm2discrete(rho_y)
    return probs
```

Funciones adicionales que calculan el kernel:

```python
self.kernel = RBFKernelLayer(sigma=sigma, 
                                    dim=encoded_size, 
                                    trainable=True)
self.qkm = QKMLayer(kernel=self.kernel, 
                                dim_x=encoded_size,
                                dim_y=dim_y, 
                                n_comp=n_comp)
```

### pure2dm

In [19]:
import tensorflow as tf

def pure2dm(psi):
    '''
    Construct a factorized density matrix to represent a pure state
    Arguments:
     psi: tensor of shape (bs, d)
    Returns:
     dm: tensor of shape (bs, 1, d + 1)
    '''
    ones = tf.ones_like(psi[:, 0:1])
    print("Columna de unos")
    print(ones)
    dm = tf.concat((ones[:, tf.newaxis, :], psi[:, tf.newaxis, :]), axis=2)
    return dm

# Crear un tensor de ejemplo para representar un estado puro
psi = tf.constant([[1.0, 0.0, 0.0], [0.0, 0.5, 0.5]], dtype=tf.float32)
print("Estado puro")
print(psi)
# Llamar a la función pure2dm para obtener la matriz de densidad factorizada
dm = pure2dm(psi)

# Imprimir la matriz de densidad resultante
print("Matriz de Densidad:")
print(dm)

# Comprobar la forma de la matriz de densidad
print("Forma de la Matriz de Densidad:", dm.shape)


Estado puro
tf.Tensor(
[[1.  0.  0. ]
 [0.  0.5 0.5]], shape=(2, 3), dtype=float32)
Columna de unos
tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
Matriz de Densidad:
tf.Tensor(
[[[1.  1.  0.  0. ]]

 [[1.  0.  0.5 0.5]]], shape=(2, 1, 4), dtype=float32)
Forma de la Matriz de Densidad: (2, 1, 4)


In [1]:
import torch
from torch import Tensor

# Definir una función para convertir un estado puro en una matriz de densidad factorizada
def pure2dm(psi: Tensor)->Tensor:
    '''
    Construct a factorized density matrix to represent a pure state
    Arguments:
     psi: tensor of shape (bs, d)
    Returns:
     dm: tensor of shape (bs, 1, d + 1)
    '''
    # Crear la columna adicional de solo unos, con el tamaño de la matriz original
    ones: Tensor = torch.ones_like(psi[:, 0:1])
    print("Columna de unos")
    print(ones)
    # Concatenar la nueva columna con la matriz original
    dm: Tensor = torch.cat((ones.unsqueeze(1), psi.unsqueeze(1)), dim=2)
    return dm

# Crear un tensor de ejemplo para representar un estado puro
psi = torch.tensor([[1.0, 0.0, 0.0], [0.0, 0.5, 0.5]], dtype=torch.float32)
print("Estado Puro:")
print(psi)

# Llamar a la función pure2dm para obtener la matriz de densidad factorizada
dm = pure2dm(psi)

# Imprimir la matriz de densidad resultante
print("Matriz de Densidad:")
print(dm)

# Comprobar la forma de la matriz de densidad
print("Forma de la Matriz de Densidad:", dm.shape)


Estado Puro:
tensor([[1.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.5000]])
Columna de unos
tensor([[1.],
        [1.]])
Matriz de Densidad:
tensor([[[1.0000, 1.0000, 0.0000, 0.0000]],

        [[1.0000, 0.0000, 0.5000, 0.5000]]])
Forma de la Matriz de Densidad: torch.Size([2, 1, 4])


### dm2discrete

In [44]:
import tensorflow as tf

def dm2comp(dm):
    '''
    Extract vectors and weights from a factorized density matrix representation
    Arguments:
     dm: tensor of shape (bs, n, d + 1)
    Returns:
     w: tensor of shape (bs, n)
     v: tensor of shape (bs, n, d)
    '''
    return dm[:, :, 0], dm[:, :, 1:]
    
def dm2discrete(dm):
    '''
    Creates a discrete distribution from the components of a density matrix
    Arguments:
     dm: tensor of shape (bs, n, d + 1)
    Returns:
     prob: vector of probabilities (bs, d)
    '''
    w, v = dm2comp(dm)
    print("Pesos"); print(w)
    print("Caracteristicas"); print(v)
    w = w / tf.reduce_sum(w, axis=-1, keepdims=True)
    print("Normalizacion de los pesos"); print(w)
    norms_v = tf.expand_dims(tf.linalg.norm(v, axis=-1), axis=-1)
    print("Normalización"); print(norms_v)
    v = v / norms_v
    print("Normalizacion bajo las caracteristicas"); print(v)
    probs = tf.einsum('...j,...ji->...i', w, v ** 2, optimize="optimal")
    return probs

# Crear una matriz de densidad de ejemplo
dm = tf.constant([[[0.3, 8.0, 0.7], 
                    [0.7, 9.0, 6.2]]], dtype=tf.float32)
print("Matriz de ejemplo")
print(dm)
# Llamar a la función dm2discrete para obtener el vector de probabilidades
probs = dm2discrete(dm)

# Imprimir el vector de probabilidades resultante
print("Vector de Probabilidades:")
print(probs)

# Comprobar la forma del vector de probabilidades
print("Forma del Vector de Probabilidades:", probs.shape)


Matriz de ejemplo
tf.Tensor(
[[[0.3 8.  0.7]
  [0.7 9.  6.2]]], shape=(1, 2, 3), dtype=float32)
Pesos
tf.Tensor([[0.3 0.7]], shape=(1, 2), dtype=float32)
Caracteristicas
tf.Tensor(
[[[8.  0.7]
  [9.  6.2]]], shape=(1, 2, 2), dtype=float32)
Normalizacion de los pesos
tf.Tensor([[0.3 0.7]], shape=(1, 2), dtype=float32)
Normalización
tf.Tensor(
[[[ 8.030566]
  [10.928861]]], shape=(1, 2, 1), dtype=float32)
Normalizacion bajo las caracteristicas
tf.Tensor(
[[[0.99619377 0.08716695]
  [0.8235076  0.5673052 ]]], shape=(1, 2, 2), dtype=float32)
Vector de Probabilidades:
tf.Tensor([[0.77243596 0.22756405]], shape=(1, 2), dtype=float32)
Forma del Vector de Probabilidades: (1, 2)


In [16]:
import torch
from torch import tensor

def dm2discrete(dm: tensor)->tensor:
    '''
    Creates a discrete distribution from the components of a density matrix
    Arguments:
     dm: tensor of shape (bs, n, d + 1)
    Returns:
     prob: vector of probabilities (bs, d)
    '''
    w, v = dm[:, :, 0], dm[:, :, 1:] # Tomamos la primera columna y su sobrante
    print("Pesos"); print(w, w.shape)
    print("Caracteristicas"); print(v)

    w = w / (torch.sum(w,dim=1, keepdim=True)) # Normalizamos los pesos
    print("Normalizacion de los pesos"); print(w, w.shape)

    norms_v = torch.norm(v,dim=-1, keepdim= True)
    print("Normalización"); print(norms_v)

    v = v/ norms_v # Normalizamos las caracterisiticas
    print("Normalizacion bajo las caracteristicas"); print(v)
    # Prodcuto componente a componente en correspondencia con la fila
    # Suma de cada columna para tener un solo vector
    produt = torch.mul(torch.transpose(w, 0 ,1), v**2); print("producto:",produt)
    probs = torch.sum(produt, dim=(0, 1)) 

    return probs

    
# Crear una matriz de densidad de ejemplo en PyTorch
dm = torch.tensor([[[0.3, 8.0, 0.7], [0.7, 9.0, 6.2]]], dtype=torch.float32)

# Imprimir la matriz de ejemplo
print("Matriz de ejemplo")
print(dm)

# Llamar a la función dm2discrete para obtener el vector de probabilidades
probs = dm2discrete(dm)

# Imprimir el vector de probabilidades resultante
print("Vector de Probabilidades:")
print(probs)

# Comprobar la forma del vector de probabilidades
print("Forma del Vector de Probabilidades:", probs.shape)

Matriz de ejemplo
tensor([[[0.3000, 8.0000, 0.7000],
         [0.7000, 9.0000, 6.2000]]])
Pesos
tensor([[0.3000, 0.7000]]) torch.Size([1, 2])
Caracteristicas
tensor([[[8.0000, 0.7000],
         [9.0000, 6.2000]]])
Normalizacion de los pesos
tensor([[0.3000, 0.7000]]) torch.Size([1, 2])
Normalización
tensor([[[ 8.0306],
         [10.9289]]])
Normalizacion bajo las caracteristicas
tensor([[[0.9962, 0.0872],
         [0.8235, 0.5673]]])
producto: tensor([[[0.2977, 0.0023],
         [0.4747, 0.2253]]])
Vector de Probabilidades:
tensor([0.7724, 0.2276])
Forma del Vector de Probabilidades: torch.Size([2])


### qkm =  QKMLayer

```python
self.qkm = QKMLayer(kernel=self.kernel, 
                                dim_x=encoded_size,
                                dim_y=dim_y, 
                                n_comp=n_comp)

```

In [ ]:
def l1_loss(vals):
    '''
    Calculate the l1 loss for a batch of vectors
    Arguments:
        vals: tensor with shape (b_size, n)
    '''
    b_size = tf.cast(tf.shape(vals)[0], dtype=tf.float32)
    vals = vals / tf.norm(vals, axis=1)[:, tf.newaxis]
    loss = tf.reduce_sum(tf.abs(vals)) / b_size
    return loss

def call(self, inputs):        
    # Weight regularizers
    if self.l1_x != 0:
        self.add_loss(self.l1_x * l1_loss(self.c_x))
    if self.l1_y != 0:
        self.add_loss(self.l1_y * l1_loss(self.c_y))
    comp_w = tf.abs(self.c_w) + 1e-6
    # normalize comp_w to sum to 1
    comp_w = comp_w / tf.reduce_sum(comp_w)
    in_w = inputs[:, :, 0]  # shape (b, n_comp_in)
    in_v = inputs[:, :, 1:] # shape (b, n_comp_in, dim_x)
    out_vw = self.kernel(in_v, self.c_x)  # shape (b, n_comp_in, n_comp)
    out_w = (tf.expand_dims(tf.expand_dims(comp_w, axis=0), axis=0) *
                tf.square(out_vw)) # shape (b, n_comp_in, n_comp)
    out_w = tf.maximum(out_w, self.eps) 
    out_w_sum = tf.reduce_sum(out_w, axis=2) # shape (b, n_comp_in)
    out_w = out_w / tf.expand_dims(out_w_sum, axis=2)
    out_w = tf.einsum('...i,...ij->...j', in_w, out_w, optimize="optimal")
            # shape (b, n_comp)
    if self.l1_act != 0:
        self.add_loss(self.l1_act * l1_loss(out_w))
    out_w = tf.expand_dims(out_w, axis=-1) # shape (b, n_comp, 1)
    out_y_shape = tf.shape(out_w) + tf.constant([0, 0, self.dim_y - 1])
    out_y = tf.broadcast_to(tf.expand_dims(self.c_y, axis=0), out_y_shape)
    out = tf.concat((out_w, out_y), 2)
    return out

In [6]:
c_w = tf.constant([[[1, 2, 3], 
                       [4, 5, 6]], 
                       [[7, 8, 9], 
                        [10, 11, 12]]])
in_w = tf.constant([[[13, 14, 15],
                      [16, 17, 18]], 
                      [[19, 20, 21], 
                       [22, 23, 24]]])

in_v= tf.constant([[[25, 26, 27], 
                    [28, 29, 30]], 
                    [[31, 32, 33], 
                     [34, 35, 36]]])


In [7]:

c_w = tf.constant([[[1, 2, 3], 
                       [4, 5, 6]], 
                       [[7, 8, 9], 
                        [10, 11, 12]]])
inputs = tf.constant([[[13, 14, 15],
                      [16, 17, 18]], 
                      [[19, 20, 21], 
                       [22, 23, 24]]])

in_v= tf.constant([[[25, 26, 27], 
                    [28, 29, 30]], 
                    [[31, 32, 33], 
                     [34, 35, 36]]])

comp_w = tf.abs(c_w) + 1e-6
# normalize comp_w to sum to 1
comp_w = comp_w / tf.reduce_sum(comp_w)
in_w = inputs[:, :, 0]  # shape (b, n_comp_in)
in_v = inputs[:, :, 1:] # shape (b, n_comp_in, dim_x)
out_vw = self.kernel(in_v, self.c_x)  # shape (b, n_comp_in, n_comp)
out_w = (tf.expand_dims(tf.expand_dims(comp_w, axis=0), axis=0) *
            tf.square(out_vw)) # shape (b, n_comp_in, n_comp)
out_w = tf.maximum(out_w, self.eps) 
out_w_sum = tf.reduce_sum(out_w, axis=2) # shape (b, n_comp_in)
out_w = out_w / tf.expand_dims(out_w_sum, axis=2)
out_w = tf.einsum('...i,...ij->...j', in_w, out_w, optimize="optimal")
        # shape (b, n_comp)
if self.l1_act != 0:
    self.add_loss(self.l1_act * l1_loss(out_w))
out_w = tf.expand_dims(out_w, axis=-1) # shape (b, n_comp, 1)
out_y_shape = tf.shape(out_w) + tf.constant([0, 0, self.dim_y - 1])
out_y = tf.broadcast_to(tf.expand_dims(self.c_y, axis=0), out_y_shape)
out = tf.concat((out_w, out_y), 2)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a int32 tensor but is a float tensor [Op:AddV2]

In [4]:
import tensorflow as tf

# Crear un tensor tridimensional ficticio
tensor = tf.constant([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])
print(tensor.shape)
# Calcular la suma a lo largo del tercer eje (axis=2)
sum_along_axis2 = tf.reduce_sum(tensor, axis=2)

# Imprimir el tensor original y el resultado de la suma
print("Tensor original:")
print(tensor.numpy())
print("\nResultado de la suma a lo largo del eje 2:")
print(sum_along_axis2.numpy())


(2, 2, 3)
Tensor original:
[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]]

Resultado de la suma a lo largo del eje 2:
[[ 6 15]
 [24 33]]


### kernel = RBFKernelLayer

```python
self.kernel = RBFKernelLayer(sigma=sigma, 
                                         dim=encoded_size, 
                                         trainable=True)
```  

In [ ]:
import tensorflow as tf
import numpy as np

# Define una función para calcular la matriz de kernel RBF
class RBFKernelLayer(tf.keras.layers.Layer):
    def __init__(self, sigma=1.0, min_sigma=1e-6):
        super(RBFKernelLayer, self).__init__()
        self.sigma = sigma
        self.min_sigma = min_sigma

    def call(self, A, B):
        '''
        Input:
            A: tensor of shape (bs, n, d)
            B: tensor of shape (m, d)
        Result:
            K: tensor of shape (bs, n, m)
        '''
        shape_A = tf.shape(A)
        shape_B = tf.shape(B)
        A_norm = tf.norm(A, axis=-1)[..., tf.newaxis] ** 2
        B_norm = tf.norm(B, axis=-1)[tf.newaxis, tf.newaxis, :] ** 2
        A_reshaped = tf.reshape(A, [-1, shape_A[2]])
        AB = tf.matmul(A_reshaped, B, transpose_b=True) 
        AB = tf.reshape(AB, [shape_A[0], shape_A[1], shape_B[0]])
        dist2 = A_norm + B_norm - 2. * AB
        dist2 = tf.clip_by_value(dist2, 0., np.inf)
        sigma = tf.clip_by_value(self.sigma, self.min_sigma, np.inf)
        K = tf.exp(-dist2 / (2. * sigma ** 2.)) # type: ignore
        return K

# Ejemplo de datos de entrada A y B
A = tf.constant(np.random.rand(2, 3, 4), dtype=tf.float32)  # (bs, n, d)
B = tf.constant(np.random.rand(5, 4), dtype=tf.float32)  # (m, d)

# Crear una instancia de la capa de kernel RBF
rbf_layer = RBFKernelLayer(sigma=0.5, min_sigma=1e-6)

# Calcular la matriz de kernel RBF
K = rbf_layer(A, B)

# Imprimir la matriz de kernel resultante
print("Matriz de Kernel RBF:")
print(K.numpy())
 

Matriz de Kernel RBF:
[[[0.21259221 0.81666887 0.27527922 0.34850094 0.79120505]
  [0.05199006 0.41257712 0.4246773  0.12295436 0.3467263 ]
  [0.1489237  0.24285115 0.7343257  0.09322954 0.20964548]]

 [[0.03198252 0.33258015 0.04884354 0.1684303  0.4052699 ]
  [0.8193839  0.47304073 0.12634103 0.78048116 0.36857757]
  [0.20553519 0.7188861  0.64144784 0.29154104 0.58870536]]]


In [17]:
import torch
import torch.nn as nn
import numpy as np

# Define una clase para calcular la matriz de kernel RBF
class RBFKernelLayer(nn.Module):
    def __init__(self, sigma=1.0, min_sigma=1e-6):
        super(RBFKernelLayer, self).__init__()
        self.sigma = sigma
        self.min_sigma = min_sigma

    def forward(self, A, B):
        '''
        Input:
            A: tensor de forma (bs, n, d)
            B: tensor de forma (m, d)
        Result:
            K: tensor de forma (bs, n, m)
        '''
        shape_A = A.size()
        shape_B = B.size()
        A_norm = torch.norm(A, dim=-1).unsqueeze(-1) ** 2
        B_norm = torch.norm(B, dim=-1).unsqueeze(0).unsqueeze(0) ** 2
        A_reshaped = A.view(-1, shape_A[2])
        AB = torch.matmul(A_reshaped, B.t())
        AB = AB.view(shape_A[0], shape_A[1], shape_B[0])
        dist2 = A_norm + B_norm - 2. * AB
        dist2 = torch.clamp(dist2, 0., float('inf'))
        sigma = torch.clamp(self.sigma, self.min_sigma, float('inf'))
        K = torch.exp(-dist2 / (2. * sigma ** 2.))
        return K

# Ejemplo de datos de entrada A y B
A = torch.tensor(np.random.rand(2, 3, 4), dtype=torch.float32)  # (bs, n, d)
B = torch.tensor(np.random.rand(5, 4), dtype=torch.float32)  # (m, d)

# Crear una instancia de la capa de kernel RBF
rbf_layer = RBFKernelLayer(sigma=0.5, min_sigma=1e-6)

# Calcular la matriz de kernel RBF
K = rbf_layer(A, B)

# Imprimir la matriz de kernel resultante
print("Matriz de Kernel RBF:")
print(K.numpy())


TypeError: clamp() received an invalid combination of arguments - got (float, float, float), but expected one of:
 * (Tensor input, Tensor min, Tensor max, *, Tensor out)
 * (Tensor input, Number min, Number max, *, Tensor out)
